#### DATA and stuffs

In [17]:
import pandas as pd
import numpy as np
import pickle
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
# for first and future models
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [13]:
# Open the pickle file for reading
with open('data4.pkl', 'rb') as file:
    # Load the data from the pickle file
    data = pickle.load(file)

# Convert the loaded data into a pandas DataFrame
df = pd.DataFrame(data)

In [25]:
pure_df =df
only_one_coff_df = df.iloc[:, :-2]
none_coff = df.iloc[:, :-3]


,acc_max_mag_1,acc_max_mag_2,acc_max_mag_3,acc_max_mag_4,acc_max_mag_5,acc_max_mag_6,acc_max_mag_7,acc_max_mag_8,acc_max_mag_9,acc_max_mag_10,...,gyr_max_mag_3,gyr_max_mag_4,gyr_max_mag_5,gyr_max_mag_6,gyr_max_mag_7,gyr_max_mag_8,gyr_max_mag_9,gyr_max_mag_10,label,FF_cwt_coeff
0,1.679561e+07,10801.345361,15443.379045,2494.648966,3506.348890,2881.347046,3416.295521,5800.178632,2368.599631,1473.222493,...,5.119511e+05,312838.119283,147240.538463,211720.022095,109374.244172,20964.153534,45291.483985,121592.068026,fall,18807.850381
1,1.666644e+07,10751.749903,22126.371544,7530.281961,6636.895371,4383.290907,5607.831278,4917.174318,1942.169619,3949.116959,...,6.657685e+05,465629.751594,116254.560780,361048.330523,248070.905590,182901.528829,45836.603978,57632.006013,fall,17766.497010
2,1.690417e+07,15663.913171,18483.969593,15089.006454,12469.690810,5080.274593,9184.062217,4675.716777,2361.136182,1662.668453,...,7.235702e+05,791065.811944,532990.905611,168062.542131,82527.888710,74912.734801,50200.552175,36780.468990,fall,17028.670636
3,1.690057e+07,27834.293492,58377.754093,9126.588154,8483.198818,5839.101415,3133.863000,1640.416392,3341.617160,4192.033963,...,6.838762e+05,380239.911926,266260.475134,219481.118060,135248.631338,136101.761931,117920.069358,73855.265233,fall,13080.791350
4,1.742747e+07,124734.902539,55886.725130,12170.351645,12544.880766,6504.401602,3993.475932,7620.558801,2822.194933,4559.995931,...,1.452541e+06,416519.314624,492788.710548,190757.846448,194313.119283,226042.230682,239743.219263,212045.393820,fall,15896.013670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,1.638037e+07,16388.013912,11433.622947,12350.902330,15188.694745,6893.254251,3960.388446,803.969797,2660.412913,1624.898955,...,7.194042e+05,389663.003137,112379.522034,80732.642351,27678.196355,53746.852251,54694.151567,19737.802113,fall,16832.910450
238,1.660464e+07,14302.864306,19305.212301,10808.249385,9077.769379,2102.045811,1453.138115,4566.588317,2296.186942,4651.806443,...,5.912486e+05,374060.032924,388659.395310,198740.942220,116755.550328,74039.095038,66162.299579,27381.467173,fall,13064.574766
239,1.612340e+07,25131.303791,5905.058076,7903.859218,7405.519345,5113.755937,3755.953231,2746.683025,1416.069762,519.482796,...,1.211368e+06,554021.651008,124375.185208,197765.321892,96612.083350,60328.711581,36068.002933,46641.073757,fall,16932.691944
240,1.613424e+07,6998.180669,15515.727474,10728.015104,4613.584971,1830.390414,812.754564,894.438233,1391.102353,1359.595058,...,1.246336e+06,233156.631661,205460.243089,112661.289028,127789.433541,80630.341534,72817.869024,21806.228030,fall,16342.926901


##### Pure

In [ ]:
# Mapping for relabeling
label_mapping = {
    'fall' :'fall',
    'rfall': 'fall',
    'lfall': 'fall',
    'light': 'fall',
    'sit': 'normal',
    'walk': 'normal',
    'step': 'normal'
}

# Replace existing labels with new labels
df['label'] = df['label'].map(label_mapping)

array(['fall', 'normal'], dtype=object)

In [64]:
# labels
y = df["label"]
prova = df.drop("label", axis=1)

# split data 
X_train, X_test, y_train, y_test = train_test_split(prova, y, test_size=0.3)

# scale the features (may be useful if we are going to add other features with different scale)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
X_test_scaled = scaler.fit_transform(X_test.astype(np.float64))

#### Logistic vs Naive Bayes

In this section, we compare the performance of two models belonging to two different categories of machine learning models: discriminative and generative.
In brief, discriminative models target either to model $p(y|x)$ directly (i.e. as logistic regression does), or to learn a direct map from the inputs to the class variable (i.e. as perceptron does).
Generative models, on the other hand, try to model $p(x|y)$ and $p(y)$ (aka the likelihood and the prior distributions) and they make predictions by using the Bayes rule taht allows to derive $p(y|x)$.


In this project, we compare the performance of logistic regression (representant of the discriminative category) and naive Bayes(representat of the generative category).
Naive Bayes models are a group of extremely fast and simple classification algorithms that are often suitable for very high-dimensional datasets. 
Naive Bayes models assume the conditional independency of the predictors, assumption which is not always true. 
Since naive Bayes models raise additional assumptions w.r.t. the logistic regression, they should be trated as a quick-and-dirty baseline for a classification problem.
In fact, if we are not dealing with particular constraints in terms of both time and information (number of observation), we should prevent fromm using this type of models.

There are many flavours of naive Bayes models but, for our purposes, we focus our attention on the Gaussian naive Bayes model. 
In this classifier, the assumption is that data from each label is drawn from a simple Gaussian distribution.

In [65]:
model = GaussianNB()
model.fit(X_train_scaled,y_train);

In [66]:
preds = model.predict(X_test_scaled)

In [67]:
accuracy_score(y_test, preds)

0.9452054794520548

As it is explained in a famous paper published by Andrew Ng et al. in 2001, it has been proved that logistic regression outperforms naive bayes with a large number of training samples.
However, the generative model reaches its asymptotic faster (O(log n)) than the discriminative model(O (n))